In [1]:
import folium
import pandas as pd
import geopandas as gpd
from folium import Choropleth
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

## Do earthquake coincide with plate boundaries ?

In [3]:
plate_boundaries = gpd.read_file('../../../datasets/Geospatial_Analysis/geospatial-learn-course-data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp')
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x:[(b, a) for (b, a) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

,HAZ_PLATES,HAZ_PLAT_1,HAZ_PLAT_2,Shape_Leng,coordinates
0,TRENCH,SERAM TROUGH (ACTIVE),6722,5.843467,"[(133.6808931800001, -5.444200361999947), (133..."
1,TRENCH,WETAR THRUST,6722,1.829013,"[(125.47879802900002, -7.760600482999962), (12..."
2,TRENCH,TRENCH WEST OF LUZON (MANILA TRENCH) NORTHERN ...,6621,6.743604,"[(120.09999798800004, 19.817899819000047), (12..."
3,TRENCH,BONIN TRENCH,9821,8.329381,"[(143.20620700100005, 26.175899215000072), (14..."
4,TRENCH,NEW GUINEA TRENCH,8001,11.998145,"[(132.8273013480001, 0.41880004000006466), (13..."


In [4]:
earthquake = pd.read_csv('../../../datasets/Geospatial_Analysis/geospatial-learn-course-data/earthquakes1970-2014.csv', parse_dates=['DateTime'])
earthquake.head()

,DateTime,Latitude,Longitude,Depth,Magnitude,MagType,NbStations,Gap,Distance,RMS,Source,EventID
0,1970-01-04 17:00:40.200,24.139,102.503,31.0,7.5,Ms,90.0,NaN,NaN,0.0,NEI,1.970010e+09
1,1970-01-06 05:35:51.800,-9.628,151.458,8.0,6.2,Ms,85.0,NaN,NaN,0.0,NEI,1.970011e+09
2,1970-01-08 17:12:39.100,-34.741,178.568,179.0,6.1,Mb,59.0,NaN,NaN,0.0,NEI,1.970011e+09
3,1970-01-10 12:07:08.600,6.825,126.737,73.0,6.1,Mb,91.0,NaN,NaN,0.0,NEI,1.970011e+09
4,1970-01-16 08:05:39.000,60.280,-152.660,85.0,6.0,ML,0.0,NaN,NaN,NaN,AK,NaN


In [5]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Your code here: Add a heatmap to the map
HeatMap(data=earthquake[['Latitude', 'Longitude']], radius=15).add_to(m_1)

# Uncomment to see a hint
q_1.a.hint()

# Show the map
embed_map(m_1, '/Users/ningxuhui/Documents/GitHub/python_all/kaggle/src/Geospatial_Analysis/03_Interactive_Maps/q_1.html')

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `folium.plugins.HeatMap()` to add a heatmap. Set `data` to a DataFrame containing the latitude and longitude locations.  We got fairly good results by setting `radius=15`. Don't forget to add it to the map with the `add_to()` method!

In [12]:
m_2 = folium.Map(location=[35, 136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)

def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darked'


for i in range(0, len(earthquake)):
    folium.Circle(
        location=[earthquake.iloc[i]['Latitude'], earthquake.iloc[i]['Longitude']],
        radius=2000,
        color=color_producer(earthquake.iloc[i]['Depth'])).add_to(m_2)

embed_map(m_2, 'q_2.html')

In [13]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file('../../../datasets/Geospatial_Analysis/geospatial-learn-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp')
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

,geometry
prefecture,
Aichi,"MULTIPOLYGON (((137.09523 34.65330, 137.09546 ..."
Akita,"MULTIPOLYGON (((139.55725 39.20330, 139.55765 ..."
Aomori,"MULTIPOLYGON (((141.39860 40.92472, 141.39806 ..."
Chiba,"MULTIPOLYGON (((139.82488 34.98967, 139.82434 ..."
Ehime,"MULTIPOLYGON (((132.55859 32.91224, 132.55904 ..."


In [14]:
population = pd.read_csv('../../../datasets/Geospatial_Analysis/geospatial-learn-course-data/japan-prefecture-population.csv')
population.set_index('prefecture', inplace=True)

area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

stats['density'] = stats['population'] / stats['area_sqkm']
stats.head()

,population,area_sqkm,density
prefecture,,,
Tokyo,12868000,1800.614782,7146.448049
Kanagawa,8943000,2383.038975,3752.771186
Osaka,8801000,1923.151529,4576.342460
Aichi,7418000,5164.400005,1436.372085
Saitama,7130000,3794.036890,1879.264806


In [15]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_3)

embed_map(m_3, 'q_3.html')

In [16]:
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)

for i in range(0,len(earthquake)):
    folium.Circle(
        location=[earthquake.iloc[i]['Latitude'], earthquake.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquake.iloc[i]['Magnitude'],
            earthquake.iloc[i]['DateTime'].year),
        radius=earthquake.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquake.iloc[i]['Magnitude'])).add_to(m_4)


embed_map(m_4, 'q_4.html')